# Rebalance Week Health

Weekly health dashboard that combines:
- Exposure path (`daily_snapshots.csv`)
- Outstanding execution (`targets.csv`)
- Turnover/slippage (`order_events.csv`, `slippage.csv`)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


## Data Loading & Panel Assembly

Loads daily snapshots, weekly targets, order events, and slippage data, then constructs a unified daily panel by joining portfolio-level outstanding order percentage, daily filled notional, and daily slippage onto the snapshot timeline. The tail preview confirms the join succeeded and all four data sources contributed correctly. This panel is the single input for all charts and the weekly health summary below.

## Rebalance Week Health — 3-Panel Chart

This 3-panel chart provides an integrated view of each rebalance week's health over the backtest. The top panel shows gross and net exposure evolving day by day; the middle panel shows what fraction of the week's orders remain outstanding each day (should trend toward zero by Friday); and the bottom panel shows daily filled notional and slippage in dollars, revealing the cost of execution on active trading days. Weeks where outstanding exposure stays elevated through Friday indicate incomplete fills that carry into the next cycle.

## Weekly Health Summary Table

This table collapses the daily panel to one row per rebalance week, reporting total order size, final outstanding percentage at end of week, and total slippage in dollars. High `final_outstanding_pct` values flag weeks where the 5-day scaling schedule failed to fully execute, which may coincide with market volatility or aggressive limit offsets causing fill failures. Use this table to identify specific problematic weeks for deeper investigation in the other trading notebooks.

In [ ]:
df_snap = read_csv_from_store('wolfpack/daily_snapshots.csv')
df_targets = read_csv_from_store('wolfpack/targets.csv')
df_events = read_csv_from_store('wolfpack/order_events.csv')
df_slippage = read_csv_from_store('wolfpack/slippage.csv')

if df_snap is None or df_targets is None:
    raise ValueError('daily_snapshots.csv and targets.csv are required.')

df_snap['date'] = pd.to_datetime(df_snap['date'])
df_targets['date'] = pd.to_datetime(df_targets['date'])
for col in ['start_w', 'weekly_target_w', 'actual_w']:
    df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce').fillna(0.0)

if df_events is not None:
    df_events['date'] = pd.to_datetime(df_events['date'])
    for col in ['fill_quantity', 'fill_price']:
        df_events[col] = pd.to_numeric(df_events[col], errors='coerce').fillna(0.0)
if df_slippage is not None:
    df_slippage['date'] = pd.to_datetime(df_slippage['date'])
    df_slippage['slippage_dollars'] = pd.to_numeric(df_slippage['slippage_dollars'], errors='coerce').fillna(0.0)

# Portfolio outstanding
dx = df_targets.copy()
dx['total_order_abs'] = (dx['weekly_target_w'] - dx['start_w']).abs()
dx['remaining_abs'] = (dx['weekly_target_w'] - dx['actual_w']).abs()
dx['remaining_abs'] = np.minimum(dx['remaining_abs'], dx['total_order_abs'])

outstanding_daily = (
    dx.groupby(['date', 'week_id'], as_index=False)
      .agg(total_order_abs=('total_order_abs', 'sum'),
           remaining_abs=('remaining_abs', 'sum'))
)
outstanding_daily['outstanding_pct'] = np.where(
    outstanding_daily['total_order_abs'] > 1e-10,
    outstanding_daily['remaining_abs'] / outstanding_daily['total_order_abs'],
    0.0
)

if df_events is not None:
    turnover_daily = (
        df_events.assign(filled_notional=(df_events['fill_quantity'].abs() * df_events['fill_price'].abs()))
                 .groupby('date', as_index=False)['filled_notional']
                 .sum()
    )
else:
    turnover_daily = pd.DataFrame(columns=['date', 'filled_notional'])

if df_slippage is not None:
    slippage_daily = (
        df_slippage.groupby('date', as_index=False)['slippage_dollars']
                   .sum()
                   .rename(columns={'slippage_dollars': 'daily_slippage_abs'})
    )
else:
    slippage_daily = pd.DataFrame(columns=['date', 'daily_slippage_abs'])

panel = df_snap.merge(outstanding_daily[['date', 'outstanding_pct']], on='date', how='left')
panel = panel.merge(turnover_daily, on='date', how='left')
panel = panel.merge(slippage_daily, on='date', how='left')
panel = panel.sort_values('date')

display(panel.tail(10))


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

axes[0].plot(panel['date'], 100 * pd.to_numeric(panel['gross_exposure'], errors='coerce'), label='Gross %', color='#1f77b4')
axes[0].plot(panel['date'], 100 * pd.to_numeric(panel['net_exposure'], errors='coerce'), label='Net %', color='#ff7f0e')
axes[0].set_title('Exposure Path')
axes[0].set_ylabel('% exposure')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(panel['date'], 100 * panel['outstanding_pct'], color='#d62728')
axes[1].set_title('Portfolio Outstanding Weekly Order %')
axes[1].set_ylabel('% outstanding')
axes[1].grid(alpha=0.3)

axes[2].bar(panel['date'], panel['filled_notional'].fillna(0.0), color='#2ca02c', alpha=0.6, label='Filled notional')
axes[2].plot(panel['date'], panel['daily_slippage_abs'].fillna(0.0), color='#9467bd', linewidth=1.5, label='Daily slippage $')
axes[2].set_title('Turnover and Slippage')
axes[2].set_ylabel('$')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
weekly_health = (
    outstanding_daily.sort_values(['week_id', 'date'])
                   .groupby('week_id', as_index=False)
                   .agg(
                       start_date=('date', 'min'),
                       end_date=('date', 'max'),
                       week_total_order=('total_order_abs', 'max'),
                       final_outstanding_pct=('outstanding_pct', 'last')
                   )
                   .sort_values('start_date')
)

if df_slippage is not None and len(df_slippage):
    slip_week = (
        df_slippage.assign(week_id=df_slippage['date'].dt.strftime('%Y-%m-%d'))
                   .groupby('week_id', as_index=False)['slippage_dollars']
                   .sum()
                   .rename(columns={'slippage_dollars': 'week_slippage_dollars'})
    )
    weekly_health = weekly_health.merge(slip_week, on='week_id', how='left')

display(weekly_health.tail(15))
